In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

### Create Dataframe

In [2]:
# NOTE: this cell was only ran once, doesn't need to be run each time
# pollutants = ['CO/', 'NO2/', 'Ozone/', 'PM2.5/', 'PM10/', 'SO2/']

# # read in files and create dataframe of all data for each pollutant
# for pollutant in pollutants:
#     # empty list to store dfs
#     dfs = []
#     # iterate through files in the directory
#     for filename in os.listdir('data/' + pollutant):
#         df = pd.read_csv('data/' + pollutant + filename)

#         if pollutant == 'PM2.5/':
#             # only keep local conditions of pm2.5
#             df = df[df["AQS Parameter Description"] == "PM2.5 - Local Conditions"]
#         dfs.append(df)

#     # concatenate all dfs in the list
#     merged_df = pd.concat(dfs, ignore_index=True)
#     df = merged_df.drop(columns=["POC", "Source", "Local Site Name", "Percent Complete", "AQS Parameter Code", "AQS Parameter Description", "Method Code", "CBSA Code", "CBSA Name", "State FIPS Code", "State", "Units"])

#     # save the merged DataFrame to a new CSV file
#     df.to_csv('data/All/all_' + pollutant[:-1] + '.csv', index=False)

In [3]:
# NOTE: feel free to edit this cell to make the data cleaner, but otherwise it doesn't need to be run
# read in dataframes for each pollutant and merge into 1
# df_CO = pd.read_csv('data/All/all_CO.csv')
# df_NO2 = pd.read_csv('data/All/all_NO2.csv')
# df_Ozone = pd.read_csv('data/All/all_Ozone.csv')
# df_PM2_5 = pd.read_csv('data/All/all_PM2.5.csv')
# df_PM10 = pd.read_csv('data/All/all_PM10.csv')
# df_SO2 = pd.read_csv('data/All/all_SO2.csv')

# cols = ['Date', 'Site ID', 'County FIPS Code', 'County', 'Site Latitude', 'Site Longitude']

# mergedCO_NO2 = df_CO.merge(df_NO2, on=cols, how="outer", suffixes=(' CO', ' NO2'))
# mergedOzone_PM2 = df_Ozone.merge(df_PM2_5, on=cols, how="outer", suffixes=(' Ozone', ' PM2.5'))
# mergedPM10_SO2 = df_PM10.merge(df_SO2, on=cols, how="outer", suffixes=(' PM10', ' SO2'))
# mergedCO_NO2_Ozone_PM2 = mergedCO_NO2.merge(mergedOzone_PM2, on=cols, how="outer")
# all_merged = mergedCO_NO2_Ozone_PM2.merge(df_PM10, on=cols, how='outer').drop(columns=["Method Description"])

# # reorder cols
# col_order = ["Date", "Site ID", "County", "County FIPS Code", "Site Latitude", "Site Longitude", 
#              "Daily Max 8-hour CO Concentration", "Daily AQI Value CO", "Daily Obs Count CO", 
#              "Daily Max 1-hour NO2 Concentration", "Daily AQI Value NO2", "Daily Obs Count NO2", 
#              "Daily Max 8-hour Ozone Concentration", "Daily AQI Value Ozone", "Daily Obs Count Ozone", 
#              "Daily Mean PM2.5 Concentration", "Daily AQI Value PM2.5", "Daily Obs Count PM2.5",
#              "Daily Mean PM10 Concentration", "Daily AQI Value", "Daily Obs Count"]
# all_merged = all_merged[col_order]
# all_merged.to_csv('data/All/all_merged.csv')

NOTE: columns are measured in different units. CO: ppm, NO2: ppb, Ozone: ppm, PM2.5: ug/m3 LC, PM10: ug/m3 SC

In [4]:
# read in dataframe
df = pd.read_csv('data/All/all_merged.csv', usecols=lambda column: "Unnamed" not in column)

In [5]:
# Display first 5 rows of data
pd.set_option('display.max_columns', None)
df.head()

,Date,Site ID,County,County FIPS Code,Site Latitude,Site Longitude,Daily Max 8-hour CO Concentration,Daily AQI Value CO,Daily Obs Count CO,Daily Max 1-hour NO2 Concentration,Daily AQI Value NO2,Daily Obs Count NO2,Daily Max 8-hour Ozone Concentration,Daily AQI Value Ozone,Daily Obs Count Ozone,Daily Mean PM2.5 Concentration,Daily AQI Value PM2.5,Daily Obs Count PM2.5,Daily Mean PM10 Concentration,Daily AQI Value,Daily Obs Count
0,01/01/2015,490030003,Box Elder,3,41.492707,-112.018863,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.0,28.0,1.0,NaN,NaN,NaN
1,01/01/2015,490037001,Box Elder,3,41.945874,-112.233973,NaN,NaN,NaN,NaN,NaN,NaN,0.036,33.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN
2,01/01/2015,490050004,Cache,5,41.731111,-111.837500,NaN,NaN,NaN,35.0,33.0,24.0,0.027,25.0,17.0,16.2,64.0,1.0,NaN,NaN,NaN
3,01/01/2015,490050004,Cache,5,41.731111,-111.837500,NaN,NaN,NaN,35.0,33.0,24.0,0.027,25.0,17.0,15.3,63.0,1.0,NaN,NaN,NaN
4,01/01/2015,490071003,Carbon,7,39.595750,-110.770111,NaN,NaN,NaN,NaN,NaN,NaN,0.045,42.0,17.0,NaN,NaN,NaN,NaN,NaN,NaN


Data Type Corrections

In [6]:
# Convert object columns to string
df[df.select_dtypes(include=['object']).columns] = df.select_dtypes(include=['object']).astype('string')

# Convert date column to datetime object
df['Date'] = pd.to_datetime(df['Date'])

Remove duplicate data

In [7]:
# Find and remove duplicated data
duplicates = df.duplicated()
print("Removing",df[duplicates].shape[0], "duplicated data points")
df = df[-df.duplicated()]

Removing 9812 duplicated data points


Feature Scaling (we may not want to do this?)

In [8]:
# Select only float columns
df_floats = df.select_dtypes(include=['float']).copy()

# Apply MinMaxScaler, allowing NaN values to persist
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df_floats), columns=df_floats.columns, index=df.index)
df = df.replace({pd.NA: np.nan})

Impute Missing Values (Warning: this cell takes roughly 20 minutes to run)

In [ ]:
# Drop rows with at least 10 missing values
df = df.dropna(thresh=df.shape[1] - 10 + 1)  # Keep rows with at least (total columns - 15 + 1) non-NaN value

# Using KNN to impute the remaining missing values in the DataFrame. 
# The highest percentage of missing values in any single column is approximately 10%, 
# making KNN a suitable imputation method.

# Initialize KNN imputer
imputer = KNNImputer(n_neighbors=3)

# Apply KNN imputation
df[df.columns] = df[df.columns].apply(pd.to_numeric, errors='coerce')
df = df.copy()  # Preserve original DataFrame
df[df.columns] = imputer.fit_transform(df[df.columns])